# Edit distance and dynamic programming

# Hamming distance

Simple implementation of Hamming distance. Function takes two strings, and returns Hamming distance between the stgings.

**assert** statement in Python halts the execution of the function if condition is not met. Since Hamming distance can be defined only for strings of equal lenght, we're using the assert statement to check if strings are the same length.

In [1]:
#simple implementation of Hamming distance between two strings

def hammingDistance(x,y):
    assert len(x) == len(y)
    hm =0
    for i in range(len(x)):
        if x[i]!=y[i]:
            hm+=1
    return hm
            

In [2]:
hammingDistance('asa', 'aba')

1

This is slight modification of hammingDistance fucntion, in order to be able to process two stings of non-equal lenght

In [3]:
#for x, y of non-equal size
def modHammingDistance(x,y):
    delta = abs(len(x)-len(y))
    hm =0
    minlen = min(len(x), len(y))
    hm = hammingDistance(x[:minlen], y[:minlen])
    return(hm+delta)

In [4]:
modHammingDistance('ab', 'aba')

1

What is the lower and upper bound on edit distance between x and y? Bellow is the function which calculates

In [5]:
def boundEditDistance(x, y):
    if x==y: return 0,0
    if len(x)==0: return len(y), len(y)
    if len(y)==0: return len(x), len(x)
    delta = abs(len(x)-len(y))
    lower = delta
    if lower==0 and x!=y:
        lower=1
    minlen = min(len(x), len(y))
    upper = hammingDistance(x[:minlen], y[:minlen]) + delta
    return lower, upper

In [6]:
boundEditDistance('create', 'creation')

(2, 3)

# Edit distance

Edit distance is defined by reccurence relation: 
> **D(i,j)=min [D(i-1,j)+1, D(i, j-1)+1, D(i-1,j-1)+𝛿(i,j)]**

and base conditions
>**D(i, 0)=i**

>**D(0,j)=j.**

We will first try to implement edit distance as recurence relation.

In [7]:
def editDistance(x,y):
    if len(x)==0: return len(y)
    if len(y)==0: return len(x)
    
    delta = 0;
    if x[-1:]==y[-1:]:
        delta = 0
    else:
        delta = 1
    
    D = editDistance(x[:-1],y) + 1
    I = editDistance(x,y[:-1]) + 1
    M = editDistance(x[:-1],y[:-1]) + delta
    
    return(min(D, I, M))

In [8]:
editDistance('xxxx','xxxx')

0

In [9]:
editDistance('baba','zaba')

1

We do know that recursions take time (and space!). Let's try to measure how much time. We're going to use jupyters magic function - **timeit**.

In [10]:
%%timeit 
editDistance('xx','xxxx')

10000 loops, best of 3: 35.4 µs per loop


In [11]:
%%timeit 
editDistance('123456789','12345678')

1 loop, best of 3: 530 ms per loop


We can see in above example how time drastically increases with small increase in string length. Since genomic data is quite large, implementing eddit distance as reccurence relation might not be so convenient. Let's try to speed it up.

Bellow is **memoEditDistance**. It's the implementation of editDistance with memoization added. When we call recurence relation on set of inputs, we memoize the result. If it happens that reccurence relation is called again with parameters we've meoized, we can skip execution and use memoized value.

In the implementation, we don't even have to memoize the exact inputs, since are top-level inputs don't change durring execution we only need to rembember if we have called reccurence relation with same input string lengths for X and Y.

In [12]:
def memoEditDistance(x,y, memo = None):
    if memo is None: memo = {}
    if len(x)==0: return len(y)
    if len(y)==0: return len(x)
    
    #we check if we have memoized this
    if (len(x), len(y)) in memo:
        return memo[(len(x), len(y))]
    
    delta = 0 if x[-1:]==y[-1:] else 1
    D = memoEditDistance(x[:-1],y, memo) + 1
    I = memoEditDistance(x,y[:-1], memo) + 1
    M = memoEditDistance(x[:-1],y[:-1], memo) + delta
    
    dist = min(D, I, M)
    
    # we add edit distance value to memoization list
    memo[(len(x), len(y))] = dist
    
    return dist

Let's use timeit again on same inputs as above. We can see significant improvement in speed. But, for genome-scale aplications, this is still not good enough.

In [13]:
%%timeit
memoEditDistance('123456789','12345678')

1000 loops, best of 3: 234 µs per loop


In [14]:
%%timeit
memoEditDistance('GCGTATGCACGC','GCTATGCCACGC')

1000 loops, best of 3: 471 µs per loop


# Edit distance - Dynamic programming approach

Here we will use bottom-up, instead of top-down approach above. Bottom down approach consist of tabular comutation via reccurence relation and traceback. 

### Tabular computation

First let's do tabulat computation. Value of each cell in the table is calculated by using recurence relation. In this case, we can easily obtain D[i-1,j] D[i, j-1], D[i-1, j-1] because their values are stored one cell up, left or diagonal from the cell which value we are computing.

We rembember that base conditions for tabular conditions are:
>**D(i, 0)=i**

>**D(0,j)=j.**

Edit distance value is stored in lower-right corner - cell (n,m). 

Write python function which takes two strngs and returns edit distance (while we're there we're allso print out the matrix).

Here we will use pythons **numphy** module. We will use numpy arrays since they are rather fast and suitable for processing of larger-scale data.

In [15]:
import numpy
def edDistDP(x,y):
    
    # here we initialize first row and first collumn with base conditions.
    D=numpy.zeros((len(x)+1, len(y)+1), dtype=int)
    D[1:,0]=range(1,len(x)+1)
    D[0,1:]=range(1,len(y)+1)
    
    
    #then we fill out the table
    for i in range(1,len(x)+1):
        for j in range(1,len(y)+1):
            sigma = 1 if x[i-1]!=y[j-1] else 0
            D[i,j]=min(D[i-1,j]+1, D[i,j-1]+1, D[i-1, j-1] + sigma)
    #print(D)
    
    #edit distance is value in the lower right cell of the distance matrix
    return (D[len(x), len(y)])

In [16]:
edDistDP('GCGTATGCACGC','GCTATGCCACGC')

2

### Tracebak

We would also need to retreive edit transcript. In order to do this we need either to backtrace pointers or to infer our way back.

Originaly, we should set pointer during computation of the table. Pointer to cell (i,j) should be set to poin to cell from which the value of the cell(i,j) was computed. 

Since in python we don't use pointer representation, we'll infer our way back by looking to the neigbour cells (D[i-1,j], D[i,j-1], D[i-1,j-1]) and deducing from which of these cells we've came to our current cell. That is which predecessor gave the minimum.

We do so by taking values of each these cells and recalculating the scores of moving from one of the cells to our current cell. Them we move to the cell with best (in case of edit distance - minimal) score. If we have several cells which have the same score, in this implementation we're prefering match/replace to insertion and insertion to deletion.
We record our movement back in order to obtain edit transcript.


**Edit transcript**

If we set reference string (x) on vertical axis and read we are comparing on horizontal axis, then:
>each horizontal movement can be interpreted as insertion - **I**

>each vertical movement can be interpreted as deletion - **D**

>each diagonal movement can be match or replacement - **M** or **R**


As we do backtrace we append edit operations to transcript string. Since edit trascript represents all edit operations moving from string start to string end, and backtrace is done going from ends to begingnning of strings, we need to revert the string we've got appending edit operations during backtrace.

In [17]:
def getTranscript(D,x,y):
    t=''
    i=len(x)
    j=len(y)
    while(i!=0 and j!=0):
        sigma = 1 if x[i-1]!=y[j-1] else 0
        Del=D[i-1,j]+1
        I=D[i,j-1]+1
        M=D[i-1, j-1]+sigma
        
        # diagonal was the best
        if M<=I and M <=Del:
            # for diagonal we check wether characters on that position were the same of different
            # based on that we add M (match) or R (replacement) edit operation
            t+='M' if sigma == 0 else 'R'
            i=i-1
            j=j-1
            
        # horizonatl was the best
        elif I<=Del:
            t+='I'
            j=j-1
            
        # diagonal was the best
        else:
            t+='D'
            i=i-1
            
    # we revert string in order to get edit transcript
    return(t[::-1])

Now we just add call for retreiving transcript:

In [18]:
import numpy
def edDistDP(x,y):
    
    # here we initialize first row and first collumn with base conditions.
    D=numpy.zeros((len(x)+1, len(y)+1), dtype=int)
    D[1:,0]=range(1,len(x)+1)
    D[0,1:]=range(1,len(y)+1)
     
    #then we fill out the table
    for i in range(1,len(x)+1):
        for j in range(1,len(y)+1):
            sigma = 1 if x[i-1]!=y[j-1] else 0
            D[i,j]=min(D[i-1,j]+1, 
                       D[i,j-1]+1, 
                       D[i-1, j-1] + sigma)
    print(D)
    #edit distance is value in the lower right cell of the distance matrix
    t = getTranscript(D,x,y)
    return (t, D[len(x), len(y)])

This way we get both edit distance and edit transcript.

In [19]:
edDistDP('GCGTATGCACGC','GCTATGCCACGC')

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12]
 [ 1  0  1  2  3  4  5  6  7  8  9 10 11]
 [ 2  1  0  1  2  3  4  5  6  7  8  9 10]
 [ 3  2  1  1  2  3  3  4  5  6  7  8  9]
 [ 4  3  2  1  2  2  3  4  5  6  7  8  9]
 [ 5  4  3  2  1  2  3  4  5  5  6  7  8]
 [ 6  5  4  3  2  1  2  3  4  5  6  7  8]
 [ 7  6  5  4  3  2  1  2  3  4  5  6  7]
 [ 8  7  6  5  4  3  2  1  2  3  4  5  6]
 [ 9  8  7  6  5  4  3  2  2  2  3  4  5]
 [10  9  8  7  6  5  4  3  2  3  2  3  4]
 [11 10  9  8  7  6  5  4  3  3  3  2  3]
 [12 11 10  9  8  7  6  5  4  4  3  3  2]]


('MMDMMMMIMMMMM', 2)

In [20]:
edDistDP('TAGAGCATAAT','TAGGATCATAT')

[[ 0  1  2  3  4  5  6  7  8  9 10 11]
 [ 1  0  1  2  3  4  5  6  7  8  9 10]
 [ 2  1  0  1  2  3  4  5  6  7  8  9]
 [ 3  2  1  0  1  2  3  4  5  6  7  8]
 [ 4  3  2  1  1  1  2  3  4  5  6  7]
 [ 5  4  3  2  1  2  2  3  4  5  6  7]
 [ 6  5  4  3  2  2  3  2  3  4  5  6]
 [ 7  6  5  4  3  2  3  3  2  3  4  5]
 [ 8  7  6  5  4  3  2  3  3  2  3  4]
 [ 9  8  7  6  5  4  3  3  3  3  2  3]
 [10  9  8  7  6  5  4  4  3  4  3  3]
 [11 10  9  8  7  6  5  5  4  3  4  3]]


('MMIMMRMMMDMM', 3)

In [24]:
edDistDP('TAGAG','TAGG')

[[0 1 2 3 4]
 [1 0 1 2 3]
 [2 1 0 1 2]
 [3 2 1 0 1]
 [4 3 2 1 1]
 [5 4 3 2 1]]


('MMMDM', 1)